In [ ]:
## Trying to build a graph to show a potential method for determining if their
##     a possibility for economy of scale, ie if we are doing two or more of the
##     same process at the same time, will we see a reduction in the time to
##     accomplish the process.
## NOTE: since the data is a randomized fake data-set, we shouldn't see it here.
##     But, the point is to show the potential to build this for this methodology.

In [ ]:
## import the postgres connector
import psycopg2

## INSERT THE PASSWORD

## establishing the connection
conn = psycopg2.connect(
   database="test_db", user='shoc', password='JustKeepSwimming', host='pg_container', port= '5432'
)

## Creating a cursor object using the cursor() method
cursor = conn.cursor()

cursor = conn.cursor()

## Executing an SQL function using the execute() method
## This function does the delta inside of the database as opposed to doing it 
##     in python.  Also this does a count of the number of same processes
##     that are occuring in the window of the process (ie, if there is one process-A
##     occuring from 0100 to 0105, then the count column will be one, but if there
##     is one at 0100 to 0110 and another at 0105 to 0111, then the count column will be
##     2)
cursor.execute("""
(select		la.image_name,	la.role,			la.user,
			la.process,
			la.start_time,	la.stop_time,
			EXTRACT(EPOCH FROM (la.stop_time - la.start_time)) AS difference,
			1 as count
from		loki_aggregated as la	left join
			loki_aggregated as la2	on	(((la.start_time < la2.start_time and la.stop_time > la2.start_time) or
										 (la.start_time < la2.stop_time and la.stop_time > la2.stop_time)) and
										 la.process = la2.process)
where		la2.image_name is null
UNION
select		la.image_name,	la.role,			la.user,
			la.process,
			la.start_time,	la.stop_time,
			EXTRACT(EPOCH FROM (la.stop_time - la.start_time)) AS difference,
			count(*) + 1 as count
from		loki_aggregated as la	left join
			loki_aggregated as la2	on	(((la.start_time < la2.start_time and la.stop_time > la2.start_time) or
										 (la.start_time < la2.stop_time and la.stop_time > la2.stop_time)) and
										 la.process = la2.process)
where		la2.image_name is not null
group by	la.image_name,	la.role,			la.user,
			la.process,
			la.start_time,	la.stop_time)
order by	start_time""")

## Fetch a single row using fetchone() method.
data = cursor.fetchall()
print("Pulled all of the data in loki_aggregated: ", type(data), " of length ", len(data))

## Closing the connection
conn.close()

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

## Make a dataframe from data
df = pd.DataFrame(data)

## For some reason when we pull from postgres we don't get column
##     names, so go ahead and add them back
df.columns =['container id', 'role', 'user name', 'process', 'start time', 'stop time', 'time delta', 'simultaneous process count']

In [ ]:
## We are going to limit ourselfs to only process-1 for the first test of this
dfP1 = df.loc[df['process'] == 'Process01']

dfP1

In [ ]:
from matplotlib import pyplot as plt

## Now we are going to build ourselves a line graph
fig, ax = plt.subplots(figsize=(20, 10))

ax.plot(dfP1['start time'], dfP1['simultaneous process count'], label = 'Number of "Process 1" occuring at the same time', color='red')
ax.tick_params(axis='y', labelcolor='red')

# Generate a new Axis instance, on the twin-X axes (same position)
ax2 = ax.twinx()

## Draw the second line (length of process) and change the scale to log
ax2.plot(dfP1['start time'], dfP1['time delta'], label = 'Time to complete "Process 1" (seconds)', color='green')
ax2.set_yscale('log')
ax2.tick_params(axis='y', labelcolor='green')

ax.legend(bbox_to_anchor=(1, 1))
ax2.legend(bbox_to_anchor=(1, .95))
plt.show()

In [ ]:
## I hated that line graph, switched to scatter plot
fig, ax = plt.subplots(figsize=(20, 10))

ax.scatter(dfP1['start time'], 
           dfP1['simultaneous process count'], 
           label = 'Number of "Process 1" occuring at the same time', color='red')
ax.tick_params(axis='y', labelcolor='red')

# Generate a new Axis instance, on the twin-X axes (same position)
ax2 = ax.twinx()

## Draw the second line (length of process) and change the scale to log
ax2.scatter(dfP1['start time'], dfP1['time delta'], label = 'Time to complete "Process 1" (seconds)', color='green')
ax2.set_yscale('log')
ax2.tick_params(axis='y', labelcolor='green')

ax.legend()
plt.show()

In [ ]:
## Using the seconds, seems to be making our chart look messed up.  Gonna add
##     another column of the time delta in minutes
timeDelta = []

for i in range(0, len(df.index)):
    timeDelta.append(float(df.loc[i, ['time delta']].to_string(header=False, index=False))/60)

## Now we are done add the time take column
df['time delta minutes'] = timeDelta
dfP1 = df.loc[df['process'] == 'Process01']

## Try our line graph again
fig, ax = plt.subplots(figsize=(20, 10))

ax.plot(dfP1['start time'], dfP1['simultaneous process count'], label = 'Number of "Process 1" occuring at the same time', color='red')
ax.tick_params(axis='y', labelcolor='red')

# Generate a new Axis instance, on the twin-X axes (same position)
ax2 = ax.twinx()

## Draw the second line (length of process) and change the scale to log
ax2.plot(dfP1['start time'], dfP1['time delta minutes'], label = 'Time to complete "Process 1" (minutes)', color='green')
ax2.tick_params(axis='y', labelcolor='green')

ax.legend(bbox_to_anchor=(1, 1))
ax2.legend(bbox_to_anchor=(1, .95))
plt.show()

In [ ]:
## Well that sucks
## Lets try a heat map

In [ ]:
import math

htMap = {}
cntMap = {}

## Since we only ever have at more 6 simultaneous processes, we'll limit
##     our array to 6
for i in range(0, len(df.index)):
    pName = df.loc[i, ['process']].to_string(header=False, index=False)
    sP = int(df.loc[i, ['simultaneous process count']].to_string(header=False, index=False))
    dltTm = float(df.loc[i, ['time delta minutes']].to_string(header=False, index=False))
    
    if not math.isnan(dltTm):    
        if not (pName in htMap.keys()):
            htMap[pName] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
            cntMap[pName] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

        ## Update the average time spent on a process
        htMap[pName][sP] = (htMap[pName][sP] * cntMap[pName][sP] + dltTm) / (cntMap[pName][sP] + 1)
        cntMap[pName][sP] += 1

## Make a data-frame from the heatmap we map
df2 = pd.DataFrame(htMap)

fig, ax = plt.subplots(figsize=(20, 10))
im = ax.imshow(df2, cmap="Greens")
fig.colorbar(im, ax=ax)

ax.set_xticks(range(len(df2.columns)))
ax.set_yticks(range(len(df2.index)))
ax.set_xticklabels(df2.columns, rotation=90)
ax.set_yticklabels(df2.index)
ax.set_xlabel("Process")
ax.set_ylabel("Simultaneous Processes")

plt.tight_layout()
plt.show()